In [ ]:
import os
import cv2
import glob
import shutil
from PIL import Image, ImageFilter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import timm

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
import torch.nn as nn
from torch.nn import functional as F
from torchvision import models
from torch.utils.data import TensorDataset, DataLoader, Dataset

import pytorch_lightning as pl
from pytorch_lightning import metrics
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger

from sklearn import metrics, model_selection, preprocessing
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarninfnet_f0sngs('ignore')

# import wandb
# !wandb login

In [ ]:
# timm.list_models()

['adv_inception_v3',
 'cspdarknet53',
 'cspdarknet53_iabn',
 'cspresnet50',
 'cspresnet50d',
 'cspresnet50w',
 'cspresnext50',
 'cspresnext50_iabn',
 'darknet53',
 'densenet121',
 'densenet121d',
 'densenet161',
 'densenet169',
 'densenet201',
 'densenet264',
 'densenet264d_iabn',
 'densenetblur121d',
 'dla34',
 'dla46_c',
 'dla46x_c',
 'dla60',
 'dla60_res2net',
 'dla60_res2next',
 'dla60x',
 'dla60x_c',
 'dla102',
 'dla102x',
 'dla102x2',
 'dla169',
 'dm_nfnet_f0',
 'dm_nfnet_f1',
 'dm_nfnet_f2',
 'dm_nfnet_f3',
 'dm_nfnet_f4',
 'dm_nfnet_f5',
 'dm_nfnet_f6',
 'dpn68',
 'dpn68b',
 'dpn92',
 'dpn98',
 'dpn107',
 'dpn131',
 'eca_vovnet39b',
 'ecaresnet26t',
 'ecaresnet50d',
 'ecaresnet50d_pruned',
 'ecaresnet50t',
 'ecaresnet101d',
 'ecaresnet101d_pruned',
 'ecaresnet200d',
 'ecaresnet269d',
 'ecaresnetlight',
 'ecaresnext26t_32x4d',
 'ecaresnext50t_32x4d',
 'efficientnet_b0',
 'efficientnet_b1',
 'efficientnet_b1_pruned',
 'efficientnet_b2',
 'efficientnet_b2_pruned',
 'efficientnet_b

In [ ]:
!unzip -q '/content/drive/MyDrive/data_sprint_31/mammography_images.zip'

replace mammography_images/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
raw_train = pd.read_csv('/content/mammography_images/Training_set.csv')
raw_test = pd.read_csv('/content/mammography_images/Testing_set.csv')

In [ ]:
print(raw_train.shape)
raw_train.head()

(5724, 2)


,filename,label
0,Image_1.jpg,Density3Benign
1,Image_2.jpg,Density1Benign
2,Image_3.jpg,Density1Malignant
3,Image_4.jpg,Density1Benign
4,Image_5.jpg,Density1Malignant


In [ ]:
print(raw_test.shape)
raw_test.head()

(1908, 1)


,filename
0,Image_1.jpg
1,Image_2.jpg
2,Image_3.jpg
3,Image_4.jpg
4,Image_5.jpg


In [ ]:
train = raw_train.copy()
train['type'] = raw_train['label'].str.extract(pat='\d(\w+)')
train['density'] = raw_train['label'].str.extract(pat='\w+(\d)\w+')

train['type'] = train['type'].replace({'Benign': 0, 'Malignant': 1})

le = LabelEncoder()
labels = le.fit_transform(train['label'].values.tolist())
train['label_enc'] = labels
train.head()

,filename,label,type,density,label_enc
0,Image_1.jpg,Density3Benign,0,3,4
1,Image_2.jpg,Density1Benign,0,1,0
2,Image_3.jpg,Density1Malignant,1,1,1
3,Image_4.jpg,Density1Benign,0,1,0
4,Image_5.jpg,Density1Malignant,1,1,1


In [ ]:
X_train = train.iloc[:4293]
X_val = train.iloc[4293:]

In [ ]:
y_train = F.one_hot(torch.tensor(X_train.label_enc.values), num_classes=8) #only for binary cross entropy loss, you don't have to one hot for cross entropy
y_val = F.one_hot(torch.tensor(X_val.label_enc.values), num_classes=8)

In [ ]:
class Data(Dataset):
    def __init__(self, dataframe, transform, target):
        super().__init__()
        self.dataframe = dataframe
        self.transform = transform
        self.target = target
    
    def __len__(self):
        return self.dataframe.shape[0]
    
    def __getitem__(self, item):
        filename = self.dataframe.iloc[item]['filename']
        img_path = os.path.join("/content/mammography_images/train", file_name)
        target = self.target[item]
        image = cv2.imread(img_path, cv2.COLOR_BGR2GRAY)

        if self.transform is not None:
        image = self.transform(image = image)['image']

        return image, torch.tensor(target)

In [ ]:
train_aug = A.Compose([
                # A.RandomResizedCrop(320,320),
                # A.RandomResizedCrop(,384), #use bigger size images, as it makes easier for you cnn to capture hidden features, and don't think that the image will become blurry, this is not a problem for cnns
                A.CLAHE(p = 0.5),
                A.Transpose(p=0.5),
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
                A.ShiftScaleRotate(p=0.5),
                # A.HueSaturationValue(
                #     hue_shift_limit=0.2, 
                #     sat_shift_limit=0.2, 
                #     val_shift_limit=0.2, 
                #     p=0.5
                # ),
                A.RandomBrightnessContrast(
                    brightness_limit=(-0.1,0.1), 
                    contrast_limit=(-0.1, 0.1), 
                    p=0.5
                ),
                A.Normalize(
                    mean=[0.485, 0.456, 0.406], 
                    std=[0.229, 0.224, 0.225], 
                    max_pixel_value=255.0, 
                    p=1.0
                ), ToTensorV2(),
])

In [ ]:
valid_aug = A.Compose([
            # A.Resize(320,320),
            # A.Resize(384,384),
            A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ), ToTensorV2(),
])

In [ ]:
class NF_Net(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super().__init__()
        self.cnn = timm.create_model(model_name='nfnet_f0s', pretrained=pretrained)
        self.classifier = nn.Sequential(
            nn.Dropout(0.1),
            nn.LeakyReLU(),
            nn.Linear(1000, 128),
            nn.LeakyReLU(),
            nn.Linear(128, num_classes),
        )
        
    def forward(self, x):
        output = self.cnn(x)
        output = self.classifier(output)
        return output

In [ ]:
class CNNClassifier(pl.LightningModule):
    def __init__(self, train_dataset, val_dataset, learning_rate=None, batch_size=16):
    super().__init__()
    self.net = NF_Net(num_classes=8)

    self.learning_rate = learning_rate
    self.batch_size = batch_size
    self.save_hyperparameters()

    self.train_acc_metric = pl.metrics.classification.Accuracy()
    self.valid_acc_metric = pl.metrics.classification.Accuracy()

    self.train_dataset = train_dataset
    self.val_dataset = val_dataset

    def forward(self, x):
    x = self.net(x)
    return x

    #use any optimizer and schedular you want (Adam and AdamW are mostly used)
    #for ReduceLROnPlateau you have to also pass the monitor metric

    def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
    #this is my go to schedular, works quite well in every scenario
    schedular = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1
        )
    # schedular = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, 
    #                                                        threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=1e-6, eps=1e-08, verbose = True)
    return {
        "optimizer":optimizer,
        "schedular":schedular,
        # "monitor":"val_loss_epoch" #for ReduceLROnPlateau
    }

    def training_step(self, batch, batch_idx):
        image, targets = batch
        y_pred = self.forward(image)
        loss = F.binary_cross_entropy_with_logits(y_pred, targets.type_as(y_pred)) #Note: Here I've used binary cross entropy with sigmoid activation loss, I also tried the cross entropy, but this seemed to converge a lot faster
        train_acc_batch = self.train_acc_metric(torch.sigmoid(y_pred), targets)
        self.log('train_acc_batch', train_acc_batch, prog_bar=True)
        self.log('train_loss_batch', loss)
        return {
            'loss': loss,
            'y_pred': y_pred,
            'y_true': targets
        }

    def training_epoch_end(self, outputs):
        current_train_loss = torch.stack([x['loss'] for x in outputs]).mean()
        train_acc = self.train_acc_metric.compute()
        print(f"Training_ACC: {train_acc}")
        self.log('train_loss_epoch', current_train_loss)
        self.log('train_acc_epoch', train_acc)

    def validation_step(self, batch, batch_idx):
        image, targets = batch
        y_pred = self.forward(image)
        loss = F.binary_cross_entropy_with_logits(y_pred, targets.type_as(y_pred))
        val_acc_batch = self.valid_acc_metric(torch.sigmoid(y_pred), targets)
        self.log('val_acc_batch', val_acc_batch)
        self.log('val_loss_batch', loss)
        return {
          'val_loss': loss,
          'y_pred': y_pred,
          'target': targets
        }

    def validation_epoch_end(self, outputs):
        current_val_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        val_acc = self.valid_acc_metric.compute()
        print(f"Validation_ACC: {val_acc}")
        self.log("val_loss_epoch", current_val_loss)
        self.log('val_acc_epoch', val_acc)
        return {"val_loss_epoch":current_val_loss}

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, pin_memory=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, pin_memory=True)

In [ ]:
# wandb_logger = WandbLogger(project='wandb-lightning', job_type='train')

train_dataset = Data(
    dataframe=X_train,
    transform=train_aug,
    target=y_train,
)

valid_dataset = Data(
    dataframe=X_val,
    transform=valid_aug,
    target=y_val,
)

checkpointer = ModelCheckpoint(
    monitor='val_loss_epoch',
    dirpath='',
    filename='nf_resnet-{epoch:02d}-{val_loss_batch:2f}',
    mode='min',
    save_top_k=1,
)

early_stopping = EarlyStopping(
    monitor='val_loss_epoch',
    patience=5,
    mode='min',
)

learning_rate_monitor = LearningRateMonitor(logging_interval='epoch')

callbacks = [checkpointer, early_stopping, learning_rate_monitor]

model = CNNClassifier(train_dataset, valid_dataset, learning_rate=5e-5, batch_size=16)

trainer = pl.Trainer(
    # auto_lr_find = True, #you can also try this to automatically find the learning rate, but in my case, this just didn't went well (the loss was diverging xD )
    # logger=wandb_logger,
    callbacks=callbacks,
    max_epochs=16,
    progress_bar_refresh_rate=20,
    gpus=1,
    # tpu_cores = 8, #for tpu, you have to also install some extra packages, a tutorial can be found on pytorch lightning docs
    accumulate_grad_batches=2, #gradient accumulation is recommended when you use bigger images, as you have to decrease the batch size to tackle the memory problem
    precision=16,
    move_metrics_to_cpu=True #by default, lightning stores metric in gpu if you have cuda enabled, making this True, will make it to save on cpu, to conserve memory on GPU  
)


# trainer.tune(model) #Uncomment if auto_lr_find is True


# print(f"Current Learning rate of the model: {model.learning_rate:.4f}")


trainer.fit(model)

torch.save(model.state_dict(), f"efficient_final.pth")

No pretrained weights exist for this model. Using random initialization.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type     | Params
----------------------------------------------
0 | net              | NF_Net   | 71.6 M
1 | train_acc_metric | Accuracy | 0     
2 | valid_acc_metric | Accuracy | 0     
----------------------------------------------
71.6 M    Trainable params
0         Non-trainable params
71.6 M    Total params
286.474   Total estimated model params size (MB)


Validation_ACC: 0.46875


Training_ACC: 0.8683612942695618


Validation_ACC: 0.8661141395568848
Training_ACC: 0.8716806173324585


Validation_ACC: 0.8705079555511475
Training_ACC: 0.8727871179580688


Validation_ACC: 0.8719941973686218


In [ ]:
torch.save(model.state_dict(), f"nf_resnet.pth")

In [ ]:
class TestData(Dataset):
    def __init__(self, dataframe, transform):
        super().__init__()
        self.dataframe = dataframe
        self.transform = transform
    
    def __len__(self):
        return self.dataframe.shape[0]
    
    def __getitem__(self, item):
      file_name = self.dataframe.iloc[item]['filename']
      img_path = os.path.join("/content/mammography_images/test", file_name)
      image = cv2.imread(img_path, cv2.COLOR_BGR2GRAY)

      if self.transform is not None:
        image = self.transform(image = image)['image']

      return image

In [ ]:
test_aug = A.Compose([
            # A.Resize(320,320),
            # A.Resize(384,384),
            A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ), ToTensorV2(),
])

In [ ]:
test_dataset = TestData(dataframe=raw_test, transform=test_aug)

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle = False)

In [ ]:
model.eval()

CNNClassifier(
  (net): NF_Net(
    (cnn): NormFreeNet(
      (stem): Sequential(
        (conv): ScaledStdConv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
        (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      )
      (stages): Sequential(
        (0): Sequential(
          (0): NormFreeBlock(
            (downsample): DownsampleAvg(
              (pool): Identity()
              (conv): ScaledStdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
            )
            (act1): ReLU()
            (conv1): ScaledStdConv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act2): ReLU(inplace=True)
            (conv2): ScaledStdConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (act3): ReLU()
            (conv3): ScaledStdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
            (drop_path): Identity()
          )
          (1): NormFreeBlock(
            (act1): ReLU()
            (conv1): 

In [ ]:
temp_preds = None
model.to("cuda")
for batch in tqdm(test_dataloader):
    images = batch.cuda()
    preds = torch.sigmoid(model(images)).cpu().detach().numpy()
    if temp_preds is None:
        temp_preds = preds
    else:
        temp_preds = np.vstack((temp_preds, preds))

In [ ]:
final_preds = np.argmax(temp_preds, axis=1)

In [ ]:
np.unique(final_preds, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([216, 539,  72, 577, 234, 144, 108,  18]))

In [ ]:
final_preds = le.inverse_transform(final_preds)

In [ ]:
sub = pd.DataFrame({
    'filename': raw_test['filename'], 'label': final_preds
})
print(sub.shape)
sub.head()

(1908, 2)


,filename,label
0,Image_1.jpg,Density3Benign
1,Image_2.jpg,Density1Benign
2,Image_3.jpg,Density2Malignant
3,Image_4.jpg,Density1Benign
4,Image_5.jpg,Density3Malignant


In [ ]:
sub.to_csv('pl_nfresnet.csv', index=False)